In [55]:
from lbl2vec import Lbl2Vec
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
import numpy as np

In [65]:
labels = ['climat', 'sport', "economie", "sciences", "technologie", "politique"]
climat_kw = ['climat', "environnement", "température", "réchauffement"]
sport_kw = ['sport', "football", "rugby", "match", "compétition", "score", "coupe", "course", "basket"]
economie_kw = ["économie", "argent", "inflation", "coût", "euro", "impôt", "dette", "croissance"]
sciences_kw = ["mathématiques", "astronomie", "lune", "espèce", "Einstein", "ingénieur", "scientifique", "science"]
technologie_kw = ["technologie", "ordinateur", 'téléphone', "serveur", "high-tech", "écran", "nanotechnologies"]
politique_kw = ["politique", "vote", "élection", "référendum", "parlement", "ministre", "débat"]
descriptive_keywords = [climat_kw, sport_kw, economie_kw, sciences_kw, technologie_kw, politique_kw]

In [ ]:
df_articles = pd.read_json('../newspaper_2.jsonl', lines=True)

In [64]:
df_articles[df_articles.category_id == 'sciences']

,article_url,title,summary,article_date,body,image_url,category_id,journal_id,scraping_date
24,https://www.20minutes.fr/sciences/2514687-2019...,Le patron d'Amazon Jeff Bezos présente son pro...,L'homme le plus riche du monde espère pouvoir ...,2019-05-10T01:24:36+02:00,"Elon Musk rêve de Mars, Jeff Bezos, lui, a la ...",https://img.20mn.fr/w7R1e97DRDKKFsCF3ZTxTg/310...,sciences,1,2022-10-23 19:19:02.044096
301,https://www.20minutes.fr/sciences/2589451-2019...,"Un fossile ressemblant à Scrat, l’écureuil de ...","L’espèce « a un museau très long et aplati, pe...",2019-08-26T11:55:22+02:00,Et si Scrat avait vraiment existé ? Un animal ...,https://img.20mn.fr/nzTcWrMOSa-OkZW43ZzLmA/310...,sciences,1,2022-10-23 19:19:30.616893
455,https://www.20minutes.fr/sciences/2586707-2019...,Australie: Des koalas nourris de matières féca...,Les microbes intestinaux des koalas les empêch...,2019-08-21T15:25:10+02:00,"Comment sauver des koalas affamés, menacés par...",https://img.20mn.fr/xX8_qqT6RJu5usmkKW-Z5g/310...,sciences,1,2022-10-23 19:19:46.198355
601,https://www.20minutes.fr/sciences/2587479-2019...,L’astéroïde Apophis ne présente « aucun risque...,"Pour Patrick Michel, directeur de recherche au...",2019-08-22T19:01:43+02:00,"Vous faites quoi, le vendredi 13 avril 2029 ? ...",https://img.20mn.fr/zbES_Wf1SPu9E9uAQ3XPeg/310...,sciences,1,2022-10-23 19:20:01.266466
781,https://www.20minutes.fr/sciences/791346-20110...,Le lancement d'Ariane 5 repoussé à cause d'un ...,None,2011-09-21T00:00:00+02:00,Le lancement de la fusée Ariane 5 qui était pr...,None,sciences,1,2022-10-23 19:20:18.746696
...,...,...,...,...,...,...,...,...,...
493080,https://www.20minutes.fr/sciences/1183999-2013...,Sida: Pas de retour du VIH chez les deux patie...,Ni chez le bébé soigné par trithérapie à sa na...,2013-07-03T00:00:00+02:00,Des nouvelles encourageantes ont été fournies ...,https://img.20mn.fr/LrJuUja7ROGhGBBuqQ53iA/310...,sciences,1,2022-10-24 07:49:19.396194
493919,https://www.20minutes.fr/sciences/1187487-2013...,La couleur d'une exoplanète révélée pour la pr...,None,2013-07-11T00:00:00+02:00,"D'un bleu cobalt profond, sa couleur rappelle ...",https://img.20mn.fr/8ytwYioMTF-vZVbYpFdi9g/310...,sciences,1,2022-10-24 07:50:28.141644
494053,https://www.20minutes.fr/sciences/1185517-2013...,Novartis: Accord avec un groupe indien sur des...,None,2013-07-08T00:00:00+02:00,Le géant pharmaceutique suisse Novartis a an...,None,sciences,1,2022-10-24 07:50:37.120332
494500,https://www.20minutes.fr/sciences/1186509-2013...,Mars: La prochaine mission de la Nasa visera à...,Curiosity a permis de déterminer que Mars avai...,2013-07-09T00:00:00+02:00,La Nasa a dévoilé mardi le cadre de sa procha...,https://img.20mn.fr/trx2YU7mRj2sfTi11Ae4ag/310...,sciences,1,2022-10-24 07:51:12.276391


In [66]:
dict_labels = {'planete': 0, 'sport': 1, 'economie': 2, 'sciences': 3, 'high-tech': 4, 'politique': 5}

p_train, p_test = train_test_split(df_articles[(df_articles.category_id == 'planete')], test_size=1000, train_size=3000)
s_train, s_test = train_test_split(df_articles[(df_articles.category_id == 'sport')], test_size=1000, train_size=3000)
e_train, e_test = train_test_split(df_articles[(df_articles.category_id == 'economie')], test_size=1000, train_size=3000)
sc_train, sc_test = train_test_split(df_articles[(df_articles.category_id == 'sciences')], test_size=500, train_size=2000)
h_train, h_test = train_test_split(df_articles[(df_articles.category_id == 'high-tech')], test_size=1000, train_size=3000)
po_train, po_test = train_test_split(df_articles[(df_articles.category_id == 'politique')], test_size=1000, train_size=3000)

train_dataset = pd.concat([p_train, s_train, e_train, sc_train, h_train, po_train])[['title', 'category_id', 'body']]
train_dataset['label'] = train_dataset.apply(lambda x: dict_labels[x['category_id']], axis=1)
test_dataset = pd.concat([p_test, s_test, e_test, sc_test, h_test, po_test])[['title', 'category_id', 'body']]
test_dataset['label'] = test_dataset.apply(lambda x: dict_labels[x['category_id']], axis=1)

In [67]:
tagged_docs = []
for index, row in tqdm(train_dataset.iterrows()):
    text = row['body'].lower().split()
    tag = row['label']
    tagged_docs.append(TaggedDocument(text, [tag]))

17000it [00:01, 14920.32it/s]


In [68]:
# init model
model = Lbl2Vec(keywords_list=descriptive_keywords, tagged_documents=tagged_docs, label_names=labels)
# train model
model.fit()

2022-12-21 17:31:25,562 - Lbl2Vec - INFO - Train document and word embeddings
2022-12-21 17:34:17,893 - Lbl2Vec - INFO - Train label embeddings
2022-12-21 17:34:17,898 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: impôt
2022-12-21 17:34:17,900 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: astronomie Einstein
2022-12-21 17:34:17,902 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: high-tech nanotechnologies


In [69]:
# get similarity scores from trained model
model.predict_model_docs()

2022-12-21 17:34:26,565 - Lbl2Vec - INFO - Get document embeddings from model
2022-12-21 17:34:26,566 - Lbl2Vec - INFO - Calculate document<->label similarities


,doc_key,most_similar_label,highest_similarity_score,climat,sport,economie,sciences,technologie,politique
0,0,climat,1.0,1.000000,-0.038134,0.164720,0.296873,0.094864,-0.009796
1,1,sport,1.0,-0.038134,1.000000,0.040454,0.130160,0.068155,0.123125
2,2,economie,1.0,0.164720,0.040454,1.000000,-0.012358,0.208165,0.174219
3,3,sciences,1.0,0.296873,0.130160,-0.012358,1.000000,0.218228,0.026183
4,4,technologie,1.0,0.094864,0.068155,0.208165,0.218228,1.000000,-0.062242
5,5,politique,1.0,-0.009796,0.123125,0.174219,0.026183,-0.062242,1.000000


In [70]:
tagged_docs_test = []
y_true = []
for index, row in tqdm(test_dataset.iterrows()):
    text = row['body'].lower().split()
    tag = str(index)
    y_true.append(row['label'])
    tagged_docs_test.append(TaggedDocument(text, [tag]))

5500it [00:00, 19528.22it/s]


In [71]:
results = model.predict_new_docs(tagged_docs=tagged_docs_test)

2022-12-21 17:34:43,643 - Lbl2Vec - INFO - Calculate document embeddings
2022-12-21 17:34:55,664 - Lbl2Vec - INFO - Calculate document<->label similarities


In [72]:
dict_labels_model = {'climat': 0, 'sport': 1, 'economie': 2, 'sciences': 3, 'technologie': 4, 'politique': 5}
r = results.most_similar_label.to_list()
y_pred = []
for lab in r:
    y_pred.append(dict_labels_model[lab])

In [73]:
i = 5
y_pred[i], y_true[i]

(4, 0)

In [74]:
acc = np.sum(np.array(y_pred) == np.array(y_true)) / len(y_pred)
acc

0.7483636363636363

In [75]:
results

,doc_key,most_similar_label,highest_similarity_score,climat,sport,economie,sciences,technologie,politique
0,396994,politique,0.149339,0.068828,0.086830,0.129872,0.025113,-0.022737,0.149339
1,344441,sciences,0.167561,0.136808,0.074881,0.104300,0.167561,0.139004,-0.049711
2,415846,sport,0.136853,0.058231,0.136853,0.130929,0.043149,0.037798,0.056022
3,418418,climat,0.223215,0.223215,0.063297,0.166818,0.191371,0.100318,0.008228
4,392436,climat,0.164021,0.164021,0.009323,0.132653,0.053662,0.044965,0.067620
...,...,...,...,...,...,...,...,...,...
5495,182178,politique,0.312493,0.031815,0.072964,0.054214,0.041979,-0.068465,0.312493
5496,439732,politique,0.301897,-0.054053,0.281410,0.036790,0.034752,0.069701,0.301897
5497,371413,politique,0.361295,-0.076978,0.065584,-0.010095,-0.032990,-0.035908,0.361295
5498,440745,politique,0.260041,0.040147,0.111507,0.068804,-0.047497,0.018877,0.260041


In [77]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(np.array(y_true), np.array(y_pred))
cm

array([[386,  82, 116, 199,  59, 158],
       [ 13, 932,  12,  10,  13,  20],
       [ 58,  56, 586,  22, 107, 171],
       [ 18,  15,   2, 436,  21,   8],
       [ 13,  30,  25,  54, 827,  51],
       [  2,  13,  28,   2,   6, 949]], dtype=int64)